<a href="https://colab.research.google.com/github/taniokah/where-is-santa/blob/master/Indexer_for_Santa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indexer for Santa

https://github.com/taniokah/icdar-wml-2019/blob/master/Image%20Search%20for%20ICDAR%20WML%202019.ipynb

In [1]:
# Crawling Santa images.

!pip install icrawler

!rm -rf google_images/*
!rm -rf bing_images/*
!rm -rf baidu_images/*

from icrawler.builtin import BaiduImageCrawler, BingImageCrawler, GoogleImageCrawler

crawler = GoogleImageCrawler(storage={"root_dir": "google_images"}, downloader_threads=4)
crawler.crawl(keyword="Santa", offset=0, max_num=1000)

bing_crawler = BingImageCrawler(storage={'root_dir': 'bing_images'}, downloader_threads=4)
bing_crawler.crawl(keyword='Santa', filters=None, offset=0, max_num=1000)

baidu_crawler = BaiduImageCrawler(storage={'root_dir': 'baidu_images'})
baidu_crawler.crawl(keyword='Santa', offset=0, max_num=1000)

MessageError: ignored

In [2]:
import os
from subprocess import Popen, PIPE, STDOUT
#%env ES_JAVA_OPTS="-Xms100m -Xmx100m"
es_server = Popen(['elasticsearch-7.5.1/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

!ps aux | grep elastic
!sleep 30
!curl -X GET "localhost:9200/"

FileNotFoundError: ignored

In [0]:
!pip install elasticsearch

from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch(timeout=20)

doc = {
    'author': 'Santa Claus',
    'text': 'Where is Santa Claus?',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", doc_type='tweet', id=1, body=doc)
print(res['result'])

res = es.get(index="test-index", doc_type='tweet', id=1)
print(res['_source'])

es.indices.refresh(index="test-index")

res = es.search(index="test-index", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

In [0]:
# Load libraries
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import sys

model = VGG16(weights='imagenet')

In [0]:
def predict(filename, featuresize, scale=1.0):
    img = image.load_img(filename, target_size=(224, 224))
    return predictimg(img, featuresize, scale=1.0)

def predictpart(filename, featuresize, scale=1.0, size=1):
    im = Image.open(filename)
    width, height = im.size
    im = im.resize((width * size, height * size))
    im_list = np.asarray(im)
    # partition
    out_img = []
    if size > 1: 
        v_split = size
        h_split = size
        [out_img.extend(np.hsplit(h_img, h_split)) for h_img in np.vsplit(im_list, v_split)]
    else:
        out_img.append(im_list)
    reslist = []
    for offset in range(size * size):
        img = Image.fromarray(out_img[offset])
        reslist.append(predictimg(img, featuresize, scale))
    return reslist

def predictimg(img, featuresize, scale=1.0):
    width, height = img.size
    img = img.resize((int(width * scale), int(height * scale)))
    img = img.resize((224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    preds = model.predict(preprocess_input(x))
    results = decode_predictions(preds, top=featuresize)[0]
    return results

def showimg(filename, title, i, scale=1.0, col=2, row=5):
    im = Image.open(filename)
    width, height = im.size
    im = im.resize((int(width * scale), int(height * scale)))
    im = im.resize((width, height))
    im_list = np.asarray(im)
    plt.subplot(col, row, i)
    plt.title(title)
    plt.axis("off")
    plt.imshow(im_list)
    
def showpartimg(filename, title, i, size, scale=1.0, col=2, row=5):
    im = Image.open(filename)
    width, height = im.size
    im = im.resize((int(width * scale), int(height * scale)))
    #im = im.resize((width, height))
    im = im.resize((width * size, height * size))
    im_list = np.asarray(im)
    # partition
    out_img = []
    if size > 1: 
        v_split = size
        h_split = size
        [out_img.extend(np.hsplit(h_img, h_split)) for h_img in np.vsplit(im_list, v_split)]
    else:
        out_img.append(im_list)
    # draw image
    for offset in range(size * size):
        im_list = out_img[offset]
        pos = i + offset
        print(str(col) + ' ' + str(row) + ' ' + str(pos))
        plt.subplot(col, row, pos)
        plt.title(title)
        plt.axis("off")
        plt.imshow(im_list)
        out_img[offset] = Image.fromarray(im_list)
    return out_img

In [4]:
# Predict an image
scale = 1.0
filename = "google_images/000001.jpg"
plt.figure(figsize=(20, 10))

#showimg(filename, "query", i+1, scale)
imgs = showpartimg(filename, "query", 1, 1, scale)
plt.show()

for img in imgs:
    reslist = predictpart(filename, 10, scale)
    for results in reslist:
        for result in results:
            print(result)
    print()

NameError: ignored

In [0]:
import math
import binascii
from datetime import datetime

def createindex(indexname):
    if es.indices.exists(index=indexname):
        es.indices.delete(index=indexname)
    es.indices.create(index=indexname,  body={
        "settings": {
            "index.mapping.total_fields.limit": 10000,
        }
    })

def loadimages(directory):
    imagefiles = []
    for file in os.listdir(directory):
        if file.rfind('.jpg') < 0:
            continue
        filepath = os.path.join(directory, file)
        imagefiles.append(filepath)
    return imagefiles

def indexfiles(indexname, directory, featuresize=10, docsize=1000):
    imagefiles = loadimages(directory)
    for i in range(len(imagefiles)):
        if i >= docsize:
            return
        filename = imagefiles[i]
        indexfile(indexname, filename, i, featuresize)
        sys.stdout.write("\r%d" % (i + 1))
        sys.stdout.flush()
    es.indices.refresh(index=indexname)    

def indexfile(indexname, filename, i, featuresize):
    rounddown = 16
    doc = {'f': filename, 's':{}, 'timestamp': datetime.now()}  # filename, synset
    results = predict(filename, featuresize) 
    l2 = float(0)
    l2_map = float(0)
    for result in results:
        score = float(str(result[2]))
        l2 = l2 + score ** 2
    for result in results:
        score = float(str(result[2]))
        synset = doc['s']
        #wnid = int(result[0][1:])
        wnid = result[0]
        #dictionary[wnid] = hex(wnid[1:])[1:]
        id = str(hex(int(wnid[1:])-2000000)[2:])
        synset[id] = {
            'i': wnid,     # wnid
            'w': result[1],   # words
            's': round(score, rounddown),         # score
            'c': round(score / math.sqrt(l2), rounddown),         # L2 norm
            'ss': round(score ** 2, rounddown),         # score^2
            #'l2': round(l2, rounddown), 
        }
    count = es.count(index=indexname, doc_type='image')['count']
    res = es.index(index=indexname, doc_type='image', id=(count + i), body=doc)

In [6]:
createindex("santa-search")

directory = "google_images/"
indexfiles("santa-search", directory, 10, 10)
directory = "bing_images/"
indexfiles("santa-search", directory, 10, 10)
directory = "baidu_images/"
indexfiles("santa-search", directory, 10, 10)

NameError: ignored

In [0]:
def searchimg(indexname, filename, num=10, topk=10, scoretype='dot', scale=1.0, partition=1):
    plt.figure(figsize=(20, 10))
    
    #showimg(filename, "query", i+1, scale)
    imgs = showpartimg(filename, "query", 1, partition, scale)
    plt.show()

    reslist = []
    for img in imgs:
        results = predictimg(img, num, scale)
        for result in results:
            print(result)
        print()
        reslist.append(search(indexname, results, num, topk, scoretype))
    return reslist

def search(indexname, synsets, num, topk, scoretype='dot', disp=True):
    base = "0.0000"
    rounddown = 4
    if scoretype == 'dot':
        inline = base
        for synset in synsets:
            score = round(synset[2], rounddown)
            if score <= 0.0:
                continue
            wnid = synset[0]
            id = str(hex(int(wnid[1:])-2000000)[2:])
            words = synset[1]
            inline += "+doc['s." + id + ".s'].value*" + str(score)
    elif scoretype == 'man':
        #inline = str(round(1/len(synsets), rounddown)) + "*(" + str(len(synsets))
        inline = str(len(synsets) / len(synsets))
        for synset in synsets:
            score = round(synset[2], rounddown)
            if score <= 0.0:
                continue
            wnid = synset[0]
            id = str(hex(int(wnid[1:])-2000000)[2:])
            words = synset[1]
            inline += "-Math.abs(doc['s." + id + ".s'].value*" + str(1/len(synsets)) + "-" + str(score / len(synsets)) + ")"
        #inline += ")"
    elif scoretype == 'var':
        l2 = float(0)
        for synset in synsets:
            score = round(synset[2], rounddown)
            if score <= 0.0:
                continue
            l2 = l2 + (1 - score ** 2)
        #inline = str(round(1/len(synsets), rounddown)) + "*(" + str(round(l2, rounddown))
        inline = str(round(l2/len(synsets), rounddown))
        for synset in synsets:
            score = round(synset[2], rounddown)
            if score <= 0.0:
                continue
            wnid = synset[0]
            id = str(hex(int(wnid[1:])-2000000)[2:])
            words = synset[1]
            inline += "+doc['s." + id + ".s'].value*" + str(round(2 * score/len(synsets), rounddown)) + "-doc['s." + id + ".ss'].value/" + str(round(len(synsets), rounddown)) 
        #inline += ")"
    elif scoretype == 'var2':
        l2 = float(0)
        for synset in synsets:
            score = round(synset[2], rounddown)
            if score <= 0.0:
                continue
            l2 = l2 + score ** 2
        #inline = str(round(1/len(synsets), rounddown)) + "*(" + str(round(l2, rounddown))
        normalize = (len(synsets)+0)*len(synsets)
        inline = str((len(synsets)+0)/len(synsets)) + str(round(-l2/normalize, rounddown))
        for synset in synsets:
            score = round(synset[2], rounddown)
            if score <= 0.0:
                continue
            wnid = synset[0]
            id = str(hex(int(wnid[1:])-2000000)[2:])
            words = synset[1]
            inline += "+doc['s." + id + ".s'].value*" + str(round(2 * score/normalize, rounddown)) + "-doc['s." + id + ".ss'].value/" + str(round(normalize, rounddown)) 
        #inline += ")"
    elif scoretype == 'cos':
        inline = base
        for synset in synsets:
            score = round(synset[2], rounddown)
            if score <= 0.0:
                continue
            wnid = synset[0]
            id = str(hex(int(wnid[1:])-2000000)[2:])
            words = synset[1]
            inline += "+doc['s." + id + ".c'].value*" + str(score)
    elif scoretype == '+cos':
        inline = base
        for synset in synsets:
            score = round(synset[2], rounddown)
            if score <= 0.0:
                continue
            wnid = synset[0]
            id = str(hex(int(wnid[1:])-2000000)[2:])
            words = synset[1]
            inline += "+doc['s." + id + ".s'].value*" + str(score)

    #print(inline)
    #return
    
    res = es.search(index=indexname, request_timeout=60, body={
        "size": topk,
        "query": {
            "function_score": {
                "query": {"match_all": {}},
                "script_score" : {"script" : {"inline": inline}}
        }}})

    if disp==True:
        print("Got " +  str(res['hits']['total']['value']) + " Hits:")
        topres = res['hits']['hits'][0:topk]
        for hit in topres:
            print(str(hit["_id"]) + " " + str(hit["_source"]["f"]) + " "  + str(hit["_score"]))
        plt.figure(figsize=(20, 10))

        for i in range(len(topres)):
            hit = topres[i]
            row = 5
            col = int(topk / 5)
            if i >= 25:
                break
            showimg(hit["_source"]["f"], hit["_id"], i+1, col, row)
        plt.show()
    
    return res

In [8]:
filename = "google_images/000338.jpg"
_ = searchimg('santa-search', filename, 10, 10, 'dot', 1.0, 1)

NameError: ignored

In [9]:
!curl -X GET "localhost:9200/"

curl: (7) Failed to connect to localhost port 9200: Connection refused


In [10]:
res = es.search(index="santa-search", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit["_source"])
    #print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

NameError: ignored